In [1]:
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map, load_checkpoint_in_model
from transformers import AutoConfig, AutoModelForCausalLM
from huggingface_hub import snapshot_download
import torch


/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# checkpoint = "facebook/opt-30b"
# checkpoint = 'intlsy/opt-175b-hyperparam'
# checkpoint = "facebook/opt-125m"
checkpoint = "facebook/opt-1.3b"

config = AutoConfig.from_pretrained(checkpoint)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

In [3]:
weights_location = snapshot_download(checkpoint, allow_patterns=["*.bin", 'pytorch_model.bin.index.json'])#hf_hub_download(checkpoint, "*.bin")

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00,  5.65it/s]


In [4]:
device_map = infer_auto_device_map(model)
device_map

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


{'': 0}

In [10]:
# m = AutoModelForCausalLM.from_pretrained(checkpoint, device_map=device_map, offload_folder='./offload_folder')
m = load_checkpoint_and_dispatch( 
    model, device_map='auto',
    offload_folder='./offload_folder',
    checkpoint=weights_location,
    max_memory={0:0, 1:0, 'cpu': '2GiB'}
)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [11]:
m.hf_device_map

{'model.decoder.embed_tokens': 'cpu',
 'model.decoder.embed_positions': 'cpu',
 'model.decoder.final_layer_norm': 'cpu',
 'model.decoder.layers.0': 'cpu',
 'model.decoder.layers.1': 'cpu',
 'model.decoder.layers.2': 'cpu',
 'model.decoder.layers.3': 'cpu',
 'model.decoder.layers.4': 'cpu',
 'model.decoder.layers.5': 'cpu',
 'model.decoder.layers.6': 'cpu',
 'model.decoder.layers.7': 'cpu',
 'model.decoder.layers.8': 'cpu',
 'model.decoder.layers.9': 'cpu',
 'model.decoder.layers.10': 'cpu',
 'model.decoder.layers.11': 'cpu',
 'model.decoder.layers.12': 'cpu',
 'model.decoder.layers.13': 'cpu',
 'model.decoder.layers.14': 'cpu',
 'model.decoder.layers.15': 'cpu',
 'model.decoder.layers.16': 'cpu',
 'model.decoder.layers.17.self_attn.k_proj': 'cpu',
 'model.decoder.layers.17.self_attn.v_proj': 'disk',
 'model.decoder.layers.17.self_attn.q_proj': 'disk',
 'model.decoder.layers.17.self_attn.out_proj': 'disk',
 'model.decoder.layers.17.activation_fn': 'disk',
 'model.decoder.layers.17.self_